In [1]:
import sys
import glob
import os, os.path
import subprocess
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import time


%matplotlib inline

In [2]:
xr.__version__

'0.10.7'

# Settings

In [3]:
casename    = 'f.e20.FHIST.f09_001'
outdir      = '/glade/u/home/lvank/scratch/archive/smbmip/' # post-fix /$CASENAME/dstGrid3d/varname will be added

varlist = 'EFLX_LH_TOT  FIRA  FIRE  FSA  FSH  FSR  QICE  QICE_MELT  QRUNOFF  QSNOMELT  QSOIL  RAIN  SNOW  TSA'.split()
#varlist = ['QICE','FSH'] # TESTING ONLY

varlist_swapsign = 'EFLX_LH_TOT FSH'.split() # sign swap

varname_albedo = 'ALBEDO' # name that will be used for re-calculated albedo 
varname_totalmelt = 'TOTAL_MELT'

do_compression = False

FILL_VALUE = 9.96921e+36

# Helper functions

In [4]:
def xr_dataset_stripped(filename):
    """
    open Xarray dataset and strip from unnecessary variables
    """
    ds = xr.open_dataset(filename)
    ds = ds.drop(('lon_bnds','lat_bnds'))
    try:
        ds = ds.drop(('lev'))
        ds = ds.drop(('elevation'))
    except ValueError:
        pass
    return ds

# Postprocessing steps

In [5]:
def compile_list_of_datasets(year):
    """
    Load all relevant files into an Xarray dataset
    """
    var_ds_list = []
    
    for varname in varlist:
        indir_var = os.path.join(outdir, casename, 'dstGrid2d', varname)
        
        files = glob.glob(os.path.join(indir_var, '*'+str(year)+'*.nc')) # all CLM vector data
        files = sorted(files)
        print("INFO: reading %s, %d files" % (varname, len(files)))
        #print(year, varname, len(files))
        assert(len(files) == 12)
        
        var_ds_list.extend([xr_dataset_stripped(filename) for filename in files])
        
    return var_ds_list

In [6]:
dpm = np.asarray(( 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)) # days per month
sec_in_day = 86400.

def convert_units_massflux(ds):
    
    for varname in varlist:
       
        if (ds[varname].units == 'mm/s'):
            print('INFO: converting units for variable %s' % varname)
            ds[varname] *= dpm[:,np.newaxis,np.newaxis]
            ds[varname] *= sec_in_day
            attrs = ds[varname].attrs
            attrs['units'] = 'mmWE/month'

In [7]:
def swap_sign(ds):
    """
    swap sign of selected variables
    to make all energy fluxes direct towards the surface
    """    
    for varname in varlist_swapsign: # global list
        
        # only do this when the variable is present and the factor hasn't been applied already
        if (varname in ds.data_vars.keys() and 
            (not 'factor_applied' in ds[varname].attrs)): 
            
            print('INFO: swapping sign for variable %s' % varname)
            ds[varname] *= -1.0
            attrs = ds[varname].attrs
            attrs['factor_applied'] = -1.0

In [8]:
def compute_albedo(ds):
    """
    compute albedo
    """
    if (not varname_albedo in ds.data_vars.keys()):
        print("INFO: computing albedo")        
        try:
            sw_tot = ds['FSA'] + ds['FSR']
            alb = ds['FSR'] / sw_tot
#            print(type(alb), alb.nbytes)
            ds[varname_albedo] = alb
            
            attrs = ds[varname_albedo].attrs
            attrs['long_name'] = 'albedo'
            attrs['units'] = '-'
        except:
            print("ERROR: unable to compute albedo")

In [9]:
def compute_melt(ds):
    """
    total melt = snow melt + ice melt
    """
    if (not varname_totalmelt in ds.data_vars.keys()):
        print("INFO: computing total melt")      
        try:
            melt = ds['QICE_MELT'] + ds['QSNOMELT']
            print(type(melt))
            ds[varname_totalmelt] = melt
            
            attrs = ds[varname_totalmelt].attrs
            attrs['long_name'] = 'Total melt (ice + snow)'
            attrs['units'] = ds['QICE_MELT'].units

        except:
            print("ERROR: unable to compute total melt")   
#            raise e

In [10]:
def improve_descriptions(ds):
    """
    make some descriptions more clear for external collaborators
    """
    try:
        attrs = ds['QICE'].attrs
        attrs['long_name'] = 'Surface mass balance'
    except:
        pass

    try:
        attrs = ds['FSH'].attrs
        attrs['long_name'] = 'sensible heat'
    except:
        pass

    try:
        attrs = ds['EFLX_LH_TOT'].attrs
        attrs['long_name'] = 'latent heat'
    except:
        pass

# Main processing loop

In [11]:
def process_year(year):
    """
    do the postprocessing for the given year
    """
    print("INFO: processing year %d" % year)
    outdir_year = os.path.join(outdir, casename, 'processed', 'pp')
    
    if not os.path.exists(outdir_year):
        os.makedirs(outdir_year) 
    
    outfile = os.path.join(outdir_year, str(year)+'_pp.nc')

    # Read all monthly datasets per variable
    var_ds_list = compile_list_of_datasets(year)
    
    # Combine all datasets into one using auto-combine
    print('INFO: merging all data')
    ts = time.time()
    ds = xr.auto_combine(var_ds_list, concat_dim='time')  
    te = time.time()
    print('INFO: merging took %.1f seconds' % (te-ts))
    print('INFO: dataset size in MB ',ds.nbytes / 1024**2)
    
    # convert units of mass variables that are defined in mm/s
    convert_units_massflux(ds)
    
    # swap sign of energy fluxes to make them direct towards the surface
    swap_sign(ds)
    
    # compute albedo
    compute_albedo(ds)
    
    # compute total melt
    compute_melt(ds)
    
    # Make variable description more clear
    improve_descriptions(ds)

    # set some global attributes
    attrs = ds.attrs
    attrs['title'] = 'CESM/CLM glacier elevation class output regridded to 3-dimensional mesh'
    attrs['institution'] ="NCAR and Utrecht University"
    attrs['contact'] = "L.vankampenhout@uu.nl"
    attrs['model'] = "CESM 2.0 in atmosphere only mode"
    attrs['comment'] = "Topography SRF_GIMP was assumed in the downscaling procedure"
    
    # set encoding for all variables
    for varname in list(ds.data_vars.keys()) + ['lat','lon']:
        ds[varname].encoding = {'dtype': 'float32', '_FillValue': FILL_VALUE}
    
    # write output
    ts = time.time()    
    if (do_compression):
        print('INFO: writing output file with compression')
        ds.to_netcdf(outfile, 
            encoding={
               varname: {'zlib': True, 'complevel': 9} for varname in (list(ds.data_vars.keys()) + ['lat','lon'])
            })   
    else:
        print('INFO: writing output file without compression')
        ds.to_netcdf(outfile) # NO COMPRESSION
    te = time.time()
    print('INFO: writing output took %.1f seconds' % (te-ts))
        
    print('INFO: done')

In [13]:
#process_year(2014)

# Loop all years

In [ ]:
#years = range(1979,2014+1)
years = range(1996,2014+1)
print(list(years))

In [ ]:
for year in years:
    process_year(year)

INFO: processing year 1979
INFO: reading EFLX_LH_TOT, 12 files
INFO: reading FIRA, 12 files
INFO: reading FIRE, 12 files
INFO: reading FSA, 12 files
INFO: reading FSH, 12 files
INFO: reading FSR, 12 files
INFO: reading QICE, 12 files
INFO: reading QICE_MELT, 12 files
INFO: reading QRUNOFF, 12 files
INFO: reading QSNOMELT, 12 files
INFO: reading QSOIL, 12 files
INFO: reading RAIN, 12 files
INFO: reading SNOW, 12 files
INFO: reading TSA, 12 files
INFO: merging all data
INFO: merging took 156.1 seconds
INFO: dataset size in MB  3177.602066040039
INFO: converting units for variable QICE
INFO: converting units for variable QICE_MELT
INFO: converting units for variable QRUNOFF
INFO: converting units for variable QSNOMELT
INFO: converting units for variable QSOIL
INFO: converting units for variable RAIN
INFO: converting units for variable SNOW
INFO: swapping sign for variable EFLX_LH_TOT
INFO: swapping sign for variable FSH
INFO: computing albedo
INFO: computing total melt
<class 'xarray.core

/glade/work/bitz/miniconda3/envs/pangeo/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


INFO: writing output took 232.5 seconds
INFO: done
INFO: processing year 1980
INFO: reading EFLX_LH_TOT, 12 files
INFO: reading FIRA, 12 files
INFO: reading FIRE, 12 files
INFO: reading FSA, 12 files
INFO: reading FSH, 12 files
INFO: reading FSR, 12 files
INFO: reading QICE, 12 files
INFO: reading QICE_MELT, 12 files
INFO: reading QRUNOFF, 12 files
INFO: reading QSNOMELT, 12 files
INFO: reading QSOIL, 12 files
INFO: reading RAIN, 12 files
INFO: reading SNOW, 12 files
INFO: reading TSA, 12 files
INFO: merging all data
INFO: merging took 143.0 seconds
INFO: dataset size in MB  3177.602066040039
INFO: converting units for variable QICE
INFO: converting units for variable QICE_MELT
INFO: converting units for variable QRUNOFF
INFO: converting units for variable QSNOMELT
INFO: converting units for variable QSOIL
INFO: converting units for variable RAIN
INFO: converting units for variable SNOW
INFO: swapping sign for variable EFLX_LH_TOT
INFO: swapping sign for variable FSH
INFO: computing al

/glade/work/bitz/miniconda3/envs/pangeo/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


INFO: writing output took 251.5 seconds
INFO: done
INFO: processing year 1981
INFO: reading EFLX_LH_TOT, 12 files
INFO: reading FIRA, 12 files
INFO: reading FIRE, 12 files
INFO: reading FSA, 12 files
INFO: reading FSH, 12 files
INFO: reading FSR, 12 files
INFO: reading QICE, 12 files
INFO: reading QICE_MELT, 12 files
INFO: reading QRUNOFF, 12 files
INFO: reading QSNOMELT, 12 files
INFO: reading QSOIL, 12 files
INFO: reading RAIN, 12 files
INFO: reading SNOW, 12 files
INFO: reading TSA, 12 files
INFO: merging all data
INFO: merging took 132.8 seconds
INFO: dataset size in MB  3177.602066040039
INFO: converting units for variable QICE
INFO: converting units for variable QICE_MELT
INFO: converting units for variable QRUNOFF
INFO: converting units for variable QSNOMELT
INFO: converting units for variable QSOIL
INFO: converting units for variable RAIN
INFO: converting units for variable SNOW
INFO: swapping sign for variable EFLX_LH_TOT
INFO: swapping sign for variable FSH
INFO: computing al

...

/glade/work/bitz/miniconda3/envs/pangeo/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")



INFO: writing output took 362.7 seconds
INFO: done
INFO: processing year 1995
INFO: reading EFLX_LH_TOT, 12 files
INFO: reading FIRA, 12 files
INFO: reading FIRE, 12 files
INFO: reading FSA, 12 files
INFO: reading FSH, 12 files
INFO: reading FSR, 12 files
INFO: reading QICE, 12 files
INFO: reading QICE_MELT, 12 files
INFO: reading QRUNOFF, 12 files
INFO: reading QSNOMELT, 12 files
INFO: reading QSOIL, 12 files
INFO: reading RAIN, 12 files
INFO: reading SNOW, 12 files
INFO: reading TSA, 12 files
INFO: merging all data
INFO: merging took 175.8 seconds
INFO: dataset size in MB  3177.602066040039
INFO: converting units for variable QICE
INFO: converting units for variable QICE_MELT
INFO: converting units for variable QRUNOFF
INFO: converting units for variable QSNOMELT
INFO: converting units for variable QSOIL
INFO: converting units for variable RAIN
INFO: converting units for variable SNOW
INFO: swapping sign for variable EFLX_LH_TOT
INFO: swapping sign for variable FSH
INFO: computing a

/glade/work/bitz/miniconda3/envs/pangeo/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


INFO: writing output took 365.0 seconds
INFO: done
INFO: processing year 1996
INFO: reading EFLX_LH_TOT, 12 files
INFO: reading FIRA, 12 files
INFO: reading FIRE, 12 files
INFO: reading FSA, 12 files
INFO: reading FSH, 12 files
INFO: reading FSR, 12 files
INFO: reading QICE, 12 files
INFO: reading QICE_MELT, 12 files
INFO: reading QRUNOFF, 12 files
INFO: reading QSNOMELT, 12 files
INFO: reading QSOIL, 12 files
INFO: reading RAIN, 12 files
INFO: reading SNOW, 12 files
INFO: reading TSA, 12 files
INFO: merging all data
INFO: merging took 256.0 seconds
INFO: dataset size in MB  3177.602066040039
INFO: converting units for variable QICE
INFO: converting units for variable QICE_MELT
INFO: converting units for variable QRUNOFF
INFO: converting units for variable QSNOMELT
INFO: converting units for variable QSOIL
INFO: converting units for variable RAIN
INFO: converting units for variable SNOW
INFO: swapping sign for variable EFLX_LH_TOT
INFO: swapping sign for variable FSH
INFO: computing al

/glade/work/bitz/miniconda3/envs/pangeo/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")
